In [1]:
import os
import pandas as pd

PATH_TO_DATA = './'

df_train_features = pd.read_csv(os.path.join(PATH_TO_DATA, 
                                             'train.csv'), 
                                    index_col='match_id_hash')

df_train_targets = pd.read_csv(os.path.join(PATH_TO_DATA, 
                                            'train_targets.csv'), 
                                   index_col='match_id_hash')

df_test_features = pd.read_csv(os.path.join(PATH_TO_DATA, 
                                            'test.csv'), 
                                   index_col='match_id_hash')

In [2]:
X = df_train_features
y = df_train_targets['radiant_win'].astype('int') #models prefer numbers instead of True/False labels
test = df_test_features

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                      test_size=0.25, random_state=17, 
                                                      shuffle=True, stratify=y)

In [9]:
#we will use Sequential model, where we can add layers easily
from tensorflow.keras.models import Sequential

#now we import different layer types to use in our model
from keras.layers import Dense, Activation, Dropout, BatchNormalization

#we will have to optimize the model, so
from keras import optimizers

#tools to control overfitting
from keras.callbacks import EarlyStopping

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
#is keras on gpu?
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

In [4]:
X.shape

(39675, 1088)

In [5]:
def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

In [10]:
model = Sequential() #we define the model

#first layer
model.add(Dense(512,input_dim=X_train.shape[1])) #basicly,245x512 matrix of weights
model.add(BatchNormalization()) #makes NN learning easier
model.add(Activation('relu')) #applying non-linear transformation
model.add(Dropout(0.2)) #say no to overfitting

#second layer
model.add(Dense(256)) #here we don't have to specify its input_dim, it knows it from previous layer automaticly
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

#third layer
model.add(Dense(256)) #here we don't have to specify its input_dim, it knows it from previous layer automaticly
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

#you can experiment with the number of layers as well...

#now the final layer to convert everything to one number(probability)
model.add(Dense(1, activation='sigmoid'))

AttributeError: module 'tensorflow' has no attribute 'get_default_graph'

In [ ]:
#Adam optimizer, there others as well, like RMSProp
adam = optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999)

#EarlyStop if things does not improve for some time
earlystop = EarlyStopping(monitor="val_auc", patience=20, verbose=1, mode='max')

In [ ]:
#Telling the model what to do...
import tensorflow as tf
model.compile(optimizer=adam,loss='binary_crossentropy',metrics = [auc])  

In [ ]:
history = model.fit(X_train, y_train, validation_data = (X_valid,y_valid),\
                       epochs=20,batch_size=64,verbose=1,callbacks=[earlystop])